<a href="https://colab.research.google.com/github/mudiojas/SDN-Data-Processing/blob/main/SDN_DDoS_ML_and_DL_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of SDN based DDoS attack using the protocols - TCP, UDP, ICMP

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [ ]:
train_data=pd.read_csv('/content/drive/MyDrive/Final Dataset/TRAIN-DATA.csv')
test_data=pd.read_csv('/content/drive/MyDrive/Final Dataset/TEST-DATA.csv')
print("Train Data")
print(train_data.head())
print("Test Data")
print(test_data.head())

Train Data
   datapath_id  version  header_length  tos  total_length  flags  offset  ttl  \
0            1        4              5    0            28      0       0  250   
1            1        4              5    0            28      0       0  250   
2            1        4              5    0            28      0       0  250   
3            1        4              5    0            28      0       0  250   
4            1        4              5    0            28      0       0  250   

   proto   csum  ... code_icmp csum_icmp  port_no  rx_bytes_ave  rx_error_ave  \
0      1  32021  ...       160     62875        1            81             0   
1      1  63527  ...        32     63339        1            78             0   
2      1  45034  ...       236     62781        1            75             0   
3      1  27057  ...       165     63140        1            73             0   
4      1  62701  ...        68     63141        1            71             0   

   rx_dropped_a

In [ ]:
def preprocess(data):
    columnsToEncode = ['src_ip', 'dst_ip']
    le = LabelEncoder()
    ohc = OneHotEncoder(handle_unknown='ignore')
    for feature in columnsToEncode:
        try:
            data[feature] = le.fit_transform(data[feature])
        except:
            print ('error ' + feature)
    return data

In [ ]:
preprocess(train_data)
train_df=pd.get_dummies(train_data, prefix=['label'])
preprocess(test_data)
test_df=pd.get_dummies(test_data, prefix=['label'])
train_df.head()

,datapath_id,version,header_length,tos,total_length,flags,offset,ttl,proto,csum,...,rx_dropped_ave,tx_bytes_ave,tx_error_ave,tx_dropped_ave,label_DDOS_ICMP,label_DDOS_TCP,label_DDOS_UDP,label_NORMAL_ICMP,label_NORMAL_TCP,label_NORMAL_UDP
0,1,4,5,0,28,0,0,250,1,32021,...,0,123,0,0,1,0,0,0,0,0
1,1,4,5,0,28,0,0,250,1,63527,...,0,123,0,0,1,0,0,0,0,0
2,1,4,5,0,28,0,0,250,1,45034,...,0,123,0,0,1,0,0,0,0,0
3,1,4,5,0,28,0,0,250,1,27057,...,0,123,0,0,1,0,0,0,0,0
4,1,4,5,0,28,0,0,250,1,62701,...,0,123,0,0,1,0,0,0,0,0


In [ ]:
y_train=train_df[["label_DDOS_ICMP","label_DDOS_TCP","label_DDOS_UDP","label_NORMAL_ICMP","label_NORMAL_TCP", "label_NORMAL_UDP"]].copy()
X_train=train_df.drop(["label_DDOS_ICMP","label_DDOS_TCP","label_DDOS_UDP","label_NORMAL_ICMP","label_NORMAL_TCP", "label_NORMAL_UDP"], axis=1)
y_test=test_df[["label_DDOS_ICMP","label_DDOS_TCP","label_DDOS_UDP","label_NORMAL_ICMP","label_NORMAL_TCP", "label_NORMAL_UDP"]].copy()
X_test=test_df.drop(["label_DDOS_ICMP","label_DDOS_TCP","label_DDOS_UDP","label_NORMAL_ICMP","label_NORMAL_TCP", "label_NORMAL_UDP"], axis=1)

# Define the columns to be dropped
columns_to_drop = ['ttl', 'src_port', 'dst_port','src_ip', 'dst_ip', 'tcp_flag', 'type_icmp', 'code_icmp', 'csum_icmp']

# Check the column names in X_train
print(X_train.columns)

# Drop the columns from X_train if they exist
X_train = X_train.drop(columns=[col for col in columns_to_drop if col in X_train.columns])
print(X_train.columns)

# Drop the columns from X_test if they exist
X_test = X_test.drop(columns=[col for col in columns_to_drop if col in X_test.columns])
# Rest of the code for model training and evaluation


print(X_test.shape)
print(X_train.shape)
print(train_df)
print(test_df)

Index(['datapath_id', 'version', 'header_length', 'tos', 'total_length',
       'flags', 'offset', 'ttl', 'proto', 'csum', 'src_ip', 'dst_ip',
       'src_port', 'dst_port', 'tcp_flag', 'type_icmp', 'code_icmp',
       'csum_icmp', 'port_no', 'rx_bytes_ave', 'rx_error_ave',
       'rx_dropped_ave', 'tx_bytes_ave', 'tx_error_ave', 'tx_dropped_ave'],
      dtype='object')
Index(['datapath_id', 'version', 'header_length', 'tos', 'total_length',
       'flags', 'offset', 'proto', 'csum', 'port_no', 'rx_bytes_ave',
       'rx_error_ave', 'rx_dropped_ave', 'tx_bytes_ave', 'tx_error_ave',
       'tx_dropped_ave'],
      dtype='object')
(180000, 16)
(420000, 16)
        datapath_id  version  header_length  tos  total_length  flags  offset  \
0                 1        4              5    0            28      0       0   
1                 1        4              5    0            28      0       0   
2                 1        4              5    0            28      0       0   
3            

In [ ]:
model = keras.Sequential([
    layers.Dense(25, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=['accuracy','CategoricalAccuracy']
)
print(X_train)

        datapath_id  version  header_length  tos  total_length  flags  offset  \
0                 1        4              5    0            28      0       0   
1                 1        4              5    0            28      0       0   
2                 1        4              5    0            28      0       0   
3                 1        4              5    0            28      0       0   
4                 1        4              5    0            28      0       0   
...             ...      ...            ...  ...           ...    ...     ...   
419995            1        4              5    0            41      0       0   
419996            1        4              5    0            41      0       0   
419997            1        4              5    0            41      0       0   
419998            1        4              5    0            41      0       0   
419999            1        4              5    0            41      0       0   

        proto   csum  port_

In [ ]:
history = model.fit(X_train, y_train,epochs=100,batch_size=2100)
accuracy_values = history.history['accuracy']

# Calculate the average accuracy
avg_accuracy = sum(accuracy_values) / len(accuracy_values)
print("Average accuracy:", avg_accuracy)
model.save('ddos_detection_model.h5')

Epoch 1/100
200/200 [==============================] - 2s 4ms/step - loss: 77.2593 - accuracy: 0.1615 - categorical_accuracy: 0.1615
Epoch 2/100
200/200 [==============================] - 1s 4ms/step - loss: 27.6535 - accuracy: 0.1692 - categorical_accuracy: 0.1692
Epoch 3/100
200/200 [==============================] - 1s 4ms/step - loss: 22.5291 - accuracy: 0.1741 - categorical_accuracy: 0.1741
Epoch 4/100
200/200 [==============================] - 1s 4ms/step - loss: 19.3615 - accuracy: 0.1965 - categorical_accuracy: 0.1965
Epoch 5/100
200/200 [==============================] - 1s 4ms/step - loss: 17.2654 - accuracy: 0.1867 - categorical_accuracy: 0.1867
Epoch 6/100
200/200 [==============================] - 1s 4ms/step - loss: 14.6498 - accuracy: 0.2010 - categorical_accuracy: 0.2010
Epoch 7/100
200/200 [==============================] - 1s 4ms/step - loss: 11.2477 - accuracy: 0.2150 - categorical_accuracy: 0.2150
Epoch 8/100
200/200 [==============================] - 1s 7ms/step - 

In [ ]:
from google.colab import files

files.download('ddos_detection_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import accuracy_score, mean_squared_error
from math import sqrt
from time import time

start = time()
y_pred=model.predict(X_test)
print(time() - start)
print(accuracy_score(np.round(y_pred),y_test))
print(sqrt(mean_squared_error(np.round(y_pred),y_test)))

5625/5625 [==============================] - 11s 2ms/step
20.612066507339478
0.9725277777777778
0.09356617787305045


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Define the RNN model
model = keras.Sequential([
    layers.Embedding(input_dim=10000, output_dim=32),
    layers.SimpleRNN(64),
    layers.Dense(6, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=2100)
accuracy_values = history.history['accuracy']

# Calculate the average accuracy
avg_accuracy = sum(accuracy_values) / len(accuracy_values)
print("Average accuracy:", avg_accuracy)

Epoch 1/10
200/200 [==============================] - 18s 80ms/step - loss: 0.1858 - accuracy: 0.9716
Epoch 2/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 3/10
200/200 [==============================] - 5s 24ms/step - loss: 9.5477e-04 - accuracy: 1.0000
Epoch 4/10
200/200 [==============================] - 5s 23ms/step - loss: 6.1581e-04 - accuracy: 1.0000
Epoch 5/10
200/200 [==============================] - 6s 28ms/step - loss: 3.7892e-04 - accuracy: 1.0000
Epoch 6/10
200/200 [==============================] - 6s 28ms/step - loss: 2.4460e-04 - accuracy: 1.0000
Epoch 7/10
200/200 [==============================] - 4s 18ms/step - loss: 1.6026e-04 - accuracy: 1.0000
Epoch 8/10
200/200 [==============================] - 5s 23ms/step - loss: 1.2710e-04 - accuracy: 1.0000
Epoch 9/10
200/200 [==============================] - 4s 19ms/step - loss: 9.2918e-05 - accuracy: 1.0000
Epoch 10/10
200/200 [==============================] - 3s 15ms

In [ ]:
accuracy_values = [1.0000, 1.0000, 1.0000, 1.0000, 0.9928, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]

avg_accuracy = sum(accuracy_values) / len(accuracy_values)
print("Average accuracy:", avg_accuracy)


Average accuracy: 0.99928


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Define the LSTM model
model = keras.Sequential([
    layers.Embedding(input_dim=10000, output_dim=32),
    layers.LSTM(64),
    layers.Dense(6, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=2100)
accuracy_values = history.history['accuracy']

# Calculate the average accuracy
avg_accuracy = sum(accuracy_values) / len(accuracy_values)
print("Average accuracy:", avg_accuracy)

Epoch 1/10
200/200 [==============================] - 18s 78ms/step - loss: 0.5348 - accuracy: 0.8294
Epoch 2/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0082 - accuracy: 0.9999
Epoch 3/10
200/200 [==============================] - 6s 28ms/step - loss: 0.0032 - accuracy: 0.9999
Epoch 4/10
200/200 [==============================] - 2s 10ms/step - loss: 0.0019 - accuracy: 0.9999
Epoch 5/10
200/200 [==============================] - 2s 9ms/step - loss: 0.0013 - accuracy: 0.9999
Epoch 6/10
200/200 [==============================] - 1s 7ms/step - loss: 9.4424e-04 - accuracy: 0.9999
Epoch 7/10
200/200 [==============================] - 2s 8ms/step - loss: 6.7011e-04 - accuracy: 1.0000
Epoch 8/10
200/200 [==============================] - 2s 11ms/step - loss: 5.1083e-04 - accuracy: 1.0000
Epoch 9/10
200/200 [==============================] - 2s 11ms/step - loss: 4.8601e-04 - accuracy: 1.0000
Epoch 10/10
200/200 [==============================] - 2s 9ms/step - loss: 3.

# Machine Learning Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the random forest classifier
rf = RandomForestClassifier()

# Train the classifier
rf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1)

print("Accuracy:", accuracy)
print("Classification Report:")
print(report)


Accuracy: 0.6666666666666666
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30000
           1       0.63      1.00      0.77     30000
           2       0.50      1.00      0.67     30000
           3       1.00      1.00      1.00     30000
           4       1.00      0.00      0.00     30000
           5       1.00      0.00      0.00     30000

   micro avg       0.72      0.67      0.69    180000
   macro avg       0.86      0.67      0.57    180000
weighted avg       0.86      0.67      0.57    180000
 samples avg       0.74      0.67      0.67    180000



In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a logistic regression classifier with increased max_iter
logistic_regression = LogisticRegression(max_iter=1000)

# Create a multi-output classifier
multi_output_classifier = MultiOutputClassifier(logistic_regression)

# Train the model
multi_output_classifier.fit(X_train_scaled, y_train)

# Make predictions
predictions = multi_output_classifier.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)

# Display accuracy for each class
for i, label in enumerate(multi_output_classifier.estimators_):
    class_accuracy = accuracy_score(y_test.iloc[:, i], predictions[:, i])
    print(f"Accuracy for class {label}: {class_accuracy:.4f}")

# Display overall accuracy
print(f"Overall accuracy: {accuracy:.4f}")

Accuracy for class LogisticRegression(max_iter=1000): 1.0000
Accuracy for class LogisticRegression(max_iter=1000): 0.9542
Accuracy for class LogisticRegression(max_iter=1000): 1.0000
Accuracy for class LogisticRegression(max_iter=1000): 1.0000
Accuracy for class LogisticRegression(max_iter=1000): 0.8333
Accuracy for class LogisticRegression(max_iter=1000): 1.0000
Overall accuracy: 0.7875


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an SVM classifier
svm_classifier = SVC()

# Create a multi-output classifier
multi_output_classifier_svm = MultiOutputClassifier(svm_classifier)

# Train the model
multi_output_classifier_svm.fit(X_train_scaled, y_train)

# Make predictions
predictions_svm = multi_output_classifier_svm.predict(X_test_scaled)

# Calculate accuracy
accuracy_svm = accuracy_score(y_test, predictions_svm)

# Display accuracy for each class
for i, label in enumerate(multi_output_classifier_svm.estimators_):
    class_accuracy = accuracy_score(y_test.iloc[:, i], predictions_svm[:, i])
    print(f"Accuracy for class {label}: {class_accuracy:.4f}")

# Display overall accuracy
print(f"Overall accuracy (SVM): {accuracy_svm:.4f}")

Accuracy for class SVC(): 1.0000
Accuracy for class SVC(): 1.0000
Accuracy for class SVC(): 1.0000
Accuracy for class SVC(): 1.0000
Accuracy for class SVC(): 1.0000
Accuracy for class SVC(): 1.0000
Overall accuracy (SVM): 1.0000


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a KNN classifier
knn_classifier = KNeighborsClassifier()

# Create a multi-output classifier
multi_output_classifier_knn = MultiOutputClassifier(knn_classifier)

# Train the model
multi_output_classifier_knn.fit(X_train_scaled, y_train)

# Make predictions
predictions_knn = multi_output_classifier_knn.predict(X_test_scaled)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, predictions_knn)

# Display accuracy for each class
for i, label in enumerate(multi_output_classifier_knn.estimators_):
    class_accuracy = accuracy_score(y_test.iloc[:, i], predictions_knn[:, i])
    print(f"Accuracy for class {label}: {class_accuracy:.4f}")

# Display overall accuracy
print(f"Overall accuracy (KNN): {accuracy_knn:.4f}")

Accuracy for class KNeighborsClassifier(): 1.0000
Accuracy for class KNeighborsClassifier(): 1.0000
Accuracy for class KNeighborsClassifier(): 1.0000
Accuracy for class KNeighborsClassifier(): 1.0000
Accuracy for class KNeighborsClassifier(): 1.0000
Accuracy for class KNeighborsClassifier(): 1.0000
Overall accuracy (KNN): 1.0000
